# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
import pandas as pd

In [ ]:
#1

tic_tac = pd.read_csv('tic-tac-toe.csv')

In [ ]:
tic_tac.info()

In [ ]:
#3

mapping = {'x':  1,
           'o': -1,
           'b':  0}

for col in ['TL','TM','TR','ML','MM','MR','BL','BM','BR']:
    tic_tac[col] = tic_tac[col].map(mapping)

tic_tac['class'] = tic_tac['class'].astype(int)

tic_tac.info()


In [ ]:
#4

X = tic_tac.iloc[:, 0:9]
y = tic_tac.iloc[:, 9]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
#1

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized,    # normalized inputs
    y,               # targets
    test_size=0.2,   # 20% test, 80% train
    random_state=42, # for reproducibility
    stratify=y       # preserve class balance
)

# quick sanity check
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# e.g. (766, 9) (192, 9) (766,) (192,)


In [ ]:
#2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 1. Instantiate
model = Sequential([
    # first hidden layer; specify input shape = 9 features
    Dense(16, activation='relu', input_shape=(9,)),
    # second hidden layer
    Dense(8, activation='relu'),
    # output layer for binary classification
    Dense(1, activation='sigmoid')
])

# 2. (Optional) view the architecture
model.summary()


In [ ]:
#3

# If you installed tensorflow:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# If you installed standalone keras:
from keras.models import Sequential
from keras.layers import Dense

# Example assumes you’ll feed in X of shape (n_samples, 9)
# and y as integer labels 0 or 1 (we’ll use sparse loss)

model = Sequential()
# Input + first hidden
model.add(Dense(32, activation='relu', input_shape=(9,)))
# additional hidden layers
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
# output layer: 2 units, Softmax to get [p(class=0), p(class=1)]
model.add(Dense(2, activation='softmax'))




In [ ]:
#4

# Compile with sparse categorical loss so you can keep y as 0/1 ints
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
#5

history = model.fit(
    X_train,         # your normalized training inputs
    y_train,         # your integer labels (0 or 1)
    epochs=30,       # number of full passes over the data
    batch_size=32,   # how many samples per gradient update
    validation_split=0.2,  # reserve 20% of X_train/y_train for validation
    verbose=1        # shows progress bar
)


In [ ]:
#6

# 1. Basic evaluation
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss:     {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# 2. (Optional) Detailed metrics
import numpy as np
from sklearn.metrics import classification_report

# get probability outputs, then pick the class with highest score
y_prob = model.predict(X_test)
y_pred = np.argmax(y_prob, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


In [ ]:
# no extra dependencies needed
model.save('tic-tac-toe.keras')


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# 1. Get predicted probabilities
y_prob = model.predict(X_test)

# 2. Turn into class labels (for softmax output)
import numpy as np
y_pred = np.argmax(y_prob, axis=1)

# 3. Compute accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {accuracy:.4f}")


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here